# Value Base

> Base class containing the core methods of CRLD agents in value space

In [ ]:
#| default_exp Agents/ValueBase

In [ ]:
#| hide
# Imports for the nbdev development environment
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import numpy as np
import itertools as it
from functools import partial

# import jax
from jax import jit
import jax.numpy as jnp
from typing import Iterable
from fastcore.utils import *

from pyCRLD.Agents.Base import abase
from pyCRLD.Utils.Helpers import *

## Strategy Functions
First, we define classes for different stragegy functions which are necessary for value-based agents. Then, we define the base class for value-based agents.

In [ ]:
#| export
class multiagent_epsilongreedy_strategy():
    """A multiagent epsilon-greedy strategy in tabular form"""
        
    def __init__(self, epsilon_greedys=None, N=None):
        """
        Policy class to create a multiagent epsilon-greedy policy.
        
        epsilon_greedys : iterable or float
            if iterable: contains exploration parameter for each agent or
            if float: contains exploration parameter for all agents 
        N : int
            number of agents, only allowed if `epsilon_greedys` is single float
        """
        egiter = hasattr(epsilon_greedys, '__iter__')
    
        if egiter:  # eps greedy iter, sm not
            self.N = len(epsilon_greedys) # Number of agents
            assert N is None, "'N' must not be specified when iterable is given"
            
        else: 
            self.N = N  # Number of agents
            assert epsilon_greedys is not None, "epsilon value must be given"
            assert type(epsilon_greedys) is float, 'Confusing parameter input'
            epsilon_greedys = [epsilon_greedys] * self.N
            
        # exploration values
        self.epsilongreedy_explorations =\
            jnp.array(epsilon_greedys).astype(float)

In [ ]:
#| export
@partial(jit, static_argnums=0)
def action_probabilities(self:multiagent_epsilongreedy_strategy, Qisa):
    """Transform Q values into epsilongreedy policy"""
    n = jnp.newaxis
    Xisa = jnp.zeros_like(Qisa)
        
    # where are the actions with maximal value?
    WhereMAXisa = Qisa == jnp.max(Qisa, axis=-1, keepdims=True)
        
    # assign 1-eps probability to max actions
    eps = self.epsilongreedy_explorations
    Xisa += (1-eps[:,n,n]) * WhereMAXisa /\
        WhereMAXisa.sum(axis=-1, keepdims=True)
        
    # assign eps probability to all actions
    Qisa_dimensions = jnp.ones_like(Qisa)
    Xisa += eps[:,n,n] * Qisa_dimensions\
        / Qisa_dimensions.sum(axis=-1, keepdims=True)
    return Xisa 

# Monkey-patching - possibly problematic, but allows seperating the function
# definition from the class definition into different cells
multiagent_epsilongreedy_strategy.action_probabilities = action_probabilities 

In [ ]:
#| export
@patch
def id(self:multiagent_epsilongreedy_strategy
       ) -> str: # id 
    """Returns an identifier to handle simulation runs."""
    id = f"j{self.__class__.__name__}_"
    for i in range(self.N):
        eg = np.array(self.epsilongreedy_explorations[i])
        id += jnp.array_str(eg, precision=5)+'_'        
    
    return id[:-1]     

## Value Base Class
Now we define the base clase for the value-based CRLD agents.

In [ ]:
#| export
class valuebase(abase):
    """
    Base class for deterministic strategy-average independent (multi-agent) reward-prediction temporal-difference reinforcement learning in value space.
    """
    
    def __init__(self,
                 env, # An environment object
                 learning_rates:Union[float, Iterable], # agents' learning rates
                 discount_factors:Union[float, Iterable], # agents' discount factors
                 strategy_function, # the strategy function object
                 choice_intensities:Union[float, Iterable]=1.0, # agents' choice intensities
                 use_prefactor=False,  # use the 1-DiscountFactor prefactor
                 opteinsum=True,  # optimize einsum functions
                 **kwargs):

        self.env = env
        Tt = env.T; assert np.allclose(Tt.sum(-1), 1)
        Rt = env.R    
        super().__init__(Tt, Rt, discount_factors, use_prefactor, opteinsum)
        self.F = jnp.array(env.F)

        # learning rates
        self.alpha = make_variable_vector(learning_rates, self.N)

        # strategy function
        assert env.N == strategy_function.N,\
            'Environment and strategy function must have the same number of\
             agents `N`'
        self.strategy_function = strategy_function

        # temporal difference error mirror (without indices)
        self.TDerror = self.RPEisa      

In [ ]:
#| export
@partial(jit, static_argnums=0)
def step(self:valuebase, 
         Qisa):  # joint state-action values
    """
    Temporal-difference reward-prediction learning step in value space,
    given joint state-action values `Qisa`.
    """
    RPisa = self.TDerror(Qisa)
    Qisa_ = Qisa + self.alpha * RPisa
    return Qisa_, RPisa
valuebase.step = step  # Monkey-patching - possibly problematic, but allows seperating the function definition from the class definition into different cells

In [ ]:
#| export
@patch
def zero_intelligence_values(self:valuebase,
                             value:float=0.0): # state-action value
    """
    Zero-intelligence causes a behavior where agents choose each action with
    equal probability.
    
    This function returns the state-action values for the zero-intelligence
    strategy with each state-action value set to `value`.
    """
    return value * jnp.ones((self.N, self.Z, self.M))

In [ ]:
#| export
@patch
def random_values(self:valuebase):
    """Returns normally distributed random state-action values."""
    return jnp.array(np.random.randn(self.N, self.Z, self.M))

In [ ]:
#| export
def id(self:valuebase
       ) -> str: # id 
    """Returns an identifier to handle simulation runs."""
    envid = self.env.id() + "__"
    agentsid = f"j{self.__class__.__name__}_"\
        + f"{str(self.alpha)}_{str(self.gamma)}_pre{self.use_prefactor}__"
    strategyid = self.strategy_function.id()
    
    return envid + agentsid + agentsid

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()